## Proyek Klasifikasi gambar


In [ ]:

import os
import zipfile
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [61]:
print(tf.__version__)

2.15.0


In [ ]:


zip_ref = zipfile.ZipFile('rockpaperscissors.zip', 'r')
zip_ref.extractall('/tmp')
zip_ref.close()


In [ ]:

base_dir = '/tmp/rockpaperscissors/rps-cv-images'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')

In [64]:
''' 
import shutil
# def
extra_classes = ['extra_class1', 'extra_class2']  # 

# Remove extra class 
for cls in extra_classes:
    cls_path = os.path.join(base_dir, cls)
    if os.path.exists(cls_path):
        shutil.rmtree(cls_path) 
        '''

" \nimport shutil\n# Potential extra class directories to remove\nextra_classes = ['extra_class1', 'extra_class2']  # Replace with the actual names\n\n# Remove extra class directories if they exist\nfor cls in extra_classes:\n    cls_path = os.path.join(base_dir, cls)\n    if os.path.exists(cls_path):\n        shutil.rmtree(cls_path) \n        "

In [65]:
import glob

image_count = len(glob.glob(base_dir + '/*/*.png'))
print(image_count)


2188


In [66]:
'''
hidden_files = glob.glob(os.path.join(base_dir, '.*')) + glob.glob(os.path.join(base_dir, '*/*.*~')) + glob.glob(os.path.join(base_dir, '*/*.ini'))

for f in hidden_files:
    print(f"Removing hidden file: {f}")
    os.remove(f)

# 
!dir \tmp\rockpaperscissors\rps-cv-images
'''

'\nhidden_files = glob.glob(os.path.join(base_dir, \'.*\')) + glob.glob(os.path.join(base_dir, \'*/*.*~\')) + glob.glob(os.path.join(base_dir, \'*/*.ini\'))\n\nfor f in hidden_files:\n    print(f"Removing hidden file: {f}")\n    os.remove(f)\n\n# Now, list the contents of the directory again to confirm\n!dir \tmp\rockpaperscissors\rps-cv-images\n'

In [ ]:
'''
for dir_name in ['train', 'val']:
    dir_path = os.path.join(base_dir, dir_name)
    if os.path.exists(dir_path):
        new_location = os.path.join('c:/tmp/rockpaperscissors', dir_name)
        shutil.move(dir_path, new_location)

# Move README file
readme_path = os.path.join(base_dir, 'README_rpc-cv-images.txt')
if os.path.exists(readme_path):
    new_location = 'c:/tmp/rockpaperscissors/README_rpc-cv-images.txt'
    shutil.move(readme_path, new_location)
'''

"\n# Move train and val directories\nfor dir_name in ['train', 'val']:\n    dir_path = os.path.join(base_dir, dir_name)\n    if os.path.exists(dir_path):\n        new_location = os.path.join('c:/tmp/rockpaperscissors', dir_name)\n        shutil.move(dir_path, new_location)\n\n# Move the README file\nreadme_path = os.path.join(base_dir, 'README_rpc-cv-images.txt')\nif os.path.exists(readme_path):\n    new_location = 'c:/tmp/rockpaperscissors/README_rpc-cv-images.txt'\n    shutil.move(readme_path, new_location)\n"

In [ ]:

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    horizontal_flip=True,
    shear_range=0.2,
    fill_mode='wrap',
    validation_split=0.4 
)

validation_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.4
)

train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = validation_datagen.flow_from_directory(
    base_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 1314 images belonging to 3 classes.
Found 874 images belonging to 3 classes.


In [ ]:

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(3, activation='softmax') 
])


model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>0.85):
            print("\nAkurasi telah mencapai >85%, training dihentikan!")
            self.model.stop_training = True


In [ ]:
callbacks = myCallback()


history = model.fit(
    train_generator,
    steps_per_epoch=25, 
    epochs=20,
    validation_data=validation_generator,
    validation_steps=5, 
    verbose=2,
    callbacks=[callbacks]
)


Epoch 1/20
25/25 - 33s - loss: 1.4544 - accuracy: 0.3805 - val_loss: 0.9946 - val_accuracy: 0.5875 - 33s/epoch - 1s/step
Epoch 2/20
25/25 - 22s - loss: 0.7984 - accuracy: 0.6688 - val_loss: 0.8828 - val_accuracy: 0.6875 - 22s/epoch - 866ms/step
Epoch 3/20
25/25 - 19s - loss: 0.4786 - accuracy: 0.8425 - val_loss: 0.2709 - val_accuracy: 0.9250 - 19s/epoch - 759ms/step
Epoch 4/20
25/25 - 22s - loss: 0.4487 - accuracy: 0.8286 - val_loss: 0.2496 - val_accuracy: 0.9187 - 22s/epoch - 881ms/step
Epoch 5/20

Akurasi telah mencapai >85%, training dihentikan!
25/25 - 21s - loss: 0.2900 - accuracy: 0.8961 - val_loss: 0.2144 - val_accuracy: 0.9250 - 21s/epoch - 844ms/step


In [72]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_18 (MaxPooli  (None, 74, 74, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_19 (Conv2D)          (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_19 (MaxPooli  (None, 36, 36, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_20 (Conv2D)          (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_20 (MaxPooli  (None, 17, 17, 128)      

In [75]:
import numpy as np
from IPython.display import display
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
from ipywidgets import FileUpload
import io

In [ ]:

upload = FileUpload(accept='.jpg,.png,.jpeg', multiple=False)
display(upload)

def on_upload_change(change):
    if not upload.value:
        return  
    
    for uploaded_filename in upload.value:
        content = upload.value[uploaded_filename]['content']
      
        img = image.load_img(io.BytesIO(content), target_size=(150,150))
  
        plt.imshow(img)
        plt.axis('off')
        plt.show()

        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x /= 255.0

        prediction = model.predict(x)
        
        print(uploaded_filename)
        predicted_class = np.argmax(prediction)  
        classes = ['rock', 'paper', 'scissors']
        print('Predicted class:', classes[predicted_class])

upload.observe(on_upload_change, names='_counter')

FileUpload(value=(), accept='.jpg,.png,.jpeg', description='Upload')

In [78]:
print(train_generator.class_indices) 

{'paper': 0, 'rock': 1, 'scissors': 2}
